In [2]:
!unzip DriverDistraction.zip


Archive:  DriverDistraction.zip
  inflating: test/img_56430.jpg      
  inflating: test/img_56431.jpg      
  inflating: test/img_56413.jpg      
  inflating: test/img_74870.jpg      
  inflating: test/img_74879.jpg      
  inflating: test/img_56432.jpg      
  inflating: test/img_56426.jpg      
  inflating: test/img_47489.jpg      
  inflating: test/img_74876.jpg      
  inflating: test/img_56414.jpg      
  inflating: test/img_47497.jpg      
  inflating: test/img_47496.jpg      
  inflating: test/img_56424.jpg      
  inflating: test/img_74868.jpg      
  inflating: test/img_56419.jpg      
  inflating: test/img_56420.jpg      
  inflating: test/img_47501.jpg      
  inflating: test/img_74875.jpg      
  inflating: test/img_47500.jpg      
  inflating: test/img_56417.jpg      
  inflating: test/img_56412.jpg      
  inflating: train/c2/img_9869.jpg   
  inflating: train/c2/img_37867.jpg  
  inflating: test/img_74869.jpg      
  inflating: test/img_23.jpg         
  inflating: train

In [7]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

# define function to load datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 10)
    return files, targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('train/')
test_files, test_targets = load_dataset('test1/')
#valid_files, valid_targets = load_dataset('dogImages/valid')


# load list of names
names = [item[17:19] for item in sorted(glob("train/*/"))]

# break training set into training and validation sets
#(train_files, valid_files) = train_files[:18000], train_files[18000:]
#(train_targets, valid_targets) = train_targets[:18000], train_targets[18000:]
train_files, valid_files, train_targets, valid_targets = train_test_split(train_files, train_targets, test_size=0.2, random_state=42)

# print statistics about the dataset

print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d total training categories.' % len(names))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

There are 2800 total images.

There are 2160 training images.
There are 10 total training categories.
There are 540 validation images.
There are 100 test images.


In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("driver_imgs_list.csv",header='infer')
print(df['classname'].head(3))
print(df.iloc[:,1].describe())
print("\n Image Counts")
print(df['classname'].value_counts(sort=False))

0    c0
1    c0
2    c0
Name: classname, dtype: object
count     22424
unique       10
top          c0
freq       2489
Name: classname, dtype: object

 Image Counts
c4    2326
c0    2489
c5    2312
c1    2267
c8    1911
c9    2129
c6    2325
c2    2317
c3    2346
c7    2002
Name: classname, dtype: int64


In [0]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [11]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255 - 0.5
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255 - 0.5
test_tensors = paths_to_tensor(test_files).astype('float32')/255 - 0.5

100%|██████████| 100/100 [00:00<00:00, 216.39it/s]


In [12]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3), kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))


model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     32896     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 256)       131328    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 256)       0         


In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
from keras.callbacks import ModelCheckpoint  

epochs = 30 # epochs used is 30 and batch size is 40

checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=40, callbacks=[checkpointer], verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 2160 samples, validate on 540 samples
Epoch 1/30





2160/2160 [==============================] - 30s 14ms/step - loss: 2.4915 - acc: 0.1412 - val_loss: 2.1546 - val_acc: 0.3037

Epoch 00001: val_loss improved from inf to 2.15459, saving model to weights.best.from_scratch.hdf5
Epoch 2/30
2160/2160 [==============================] - 15s 7ms/step - loss: 1.5718 - acc: 0.4736 - val_loss: 0.9418 - val_acc: 0.6981

Epoch 00002: val_loss improved from 2.15459 to 0.94177, saving model to weights.best.from_scratch.hdf5
Epoch 3/30
2160/2160 [==============================] - 15s 7ms/step - loss: 0.8725 - acc: 0.7144 - val_loss: 0.4637 - val_acc: 0.8500

Epoch 00003: val_loss improved from 0.94177 to 0.46373, saving model to weights.best.from_scratch.hdf5
Epoch 4/30
2160/2160 [==============================] - 15s 7ms/step - loss: 0.5159 - acc: 0.8426 - val_loss: 0.3225 - val_acc: 0.8907

E

In [0]:
model.load_weights('weights.best.from_scratch.hdf5')

In [0]:
test_files_final = [item_test[15:] for item_test in test_files]

In [0]:
predictions = [model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in test_tensors]

In [0]:
subm = np.column_stack((np.asarray(test_files_final), np.asarray(predictions,dtype=np.float32)))

In [19]:
print(subm[1:4])

[['56419.jpg' '0.0022895127' '2.8408667e-07' '7.8805606e-08' '0.8331661'
  '0.14779209' '0.0016431644' '0.00016255464' '1.9285779e-05'
  '1.0019339e-05' '0.014916954']
 ['12.jpg' '4.9897367e-06' '0.999845' '1.6419705e-07' '9.488597e-06'
  '8.918776e-07' '1.3992445e-09' '2.0738902e-05' '3.779112e-06'
  '0.00011424241' '6.1229787e-07']
 ['42757.jpg' '1.3967099e-07' '0.0003335617' '6.8991816e-05'
  '8.0807624e-08' '1.0488196e-05' '3.3451273e-09' '0.9970873'
  '7.3504366e-06' '0.0024920406' '1.5581834e-08']]
